<a href="https://www.kaggle.com/code/barramalikwibowo/model-version-0-1?scriptVersionId=130776322" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import Adam
import os
from keras.applications import VGG16, ResNet50, MobileNet

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# grader-required-cell

# Define the training and validation base directories
train_dir = '/kaggle/input/tes2plant/Version2/Train'
validation_dir = '/kaggle/input/tes2plant/Version2/Valid'


# Check the number of images for each class and set
print(f"There are {len(os.listdir(train_dir))} class for training.\n")
print(f"There are {len(os.listdir(validation_dir))} class for validation.\n")

There are 32 class for training.

There are 32 class for validation.



In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')  
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                      batch_size=10, 
                                                      class_mode='categorical',
                                                      target_size=(224, 224))

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  # Remember that validation data should not be augmented
validation_datagen = ImageDataGenerator(rescale=1./255)

  # Pass in the appropriate arguments to the flow_from_directory method
validation_generator = validation_datagen.flow_from_directory(directory=validation_dir,
                                                                batch_size=10, 
                                                                class_mode='categorical',
                                                                target_size=(224, 224))
  ### END CODE HERE
  

Found 55274 images belonging to 32 classes.
Found 13822 images belonging to 32 classes.


In [4]:
# Load Inception model without top (fully connected) layers
base_model = InceptionV3(weights='imagenet', include_top=False)
for each_layer in base_model.layers:
        each_layer.trainable=False
# Add your own top (fully connected) layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x= Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x= Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x= Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x= Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x= Dropout(0.5)(x)
predictions = Dense(32, activation='softmax')(x) # Replace num_classes with the number of classes in your task

# Create the transfer learning model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

87910968/87910968 [==============================] - 0s 0us/step


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                     32)                                                           

In [ ]:
# Train the model
model.fit(train_generator,
                    steps_per_epoch=100,
                    epochs=100,
                    validation_data=validation_generator,
                    validation_steps=100)

In [ ]:
#model.save('mymodel')

In [ ]:

''''uploaded = files.upload()
for fn in uploaded.keys():
  # predict images
    path = fn
    img_source = image.load_img(path, target_size = (224, 224))
    imgplot = plt.imshow(img_source)
    x = image.img_to_array(img_source)
    x = x / 225.
    x = np.expand_dims(x, axis = 0)

    test_data = np.vstack([x])
    classes = model.predict(test_data)

    print(fn)
    result = np.where(classes == np.amax(classes))
    if result[1] == 0:
        print('Blueberry___healthy')
    elif result[1] == 1:
        print('Kaca')
    elif result[1] == 2:
        print('Kertas - Karton')
    elif result[1] == 3:
        print('Logam')
    elif result[1] == 4:
        print('Plastik')
    elif result[1] == 5:
        print('Sisa Makanan')
    elif result[1] == 6:
        print('Tekstil')
    elif result[1] == 7:
        print('Tekstil')
    elif result[1] == 8:
        print('Tekstil')
    elif result[1] == 9:
        print('Tekstil')
    elif result[1] == 10:
        print('Tekstil')
    elif result[1] == 11:
        print('Tekstil')
    elif result[1] == 12:
        print('Tekstil')
    elif result[1] == 13:
        print('Tekstil')
    elif result[1] == 14:
        print('Tekstil')


In [ ]:
''''import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Set path to the image file
img_path = '/kaggle/input/testing/Rust_blueberry.jpg'

# Load the image and preprocess it
img = load_img(img_path, target_size=(224, 224))
img = img_to_array(img)
img = np.expand_dims(img, axis=0)
img = tf.keras.applications.resnet50.preprocess_input(img)

# Load the trained model
model = tf.keras.models.load_model('/kaggle/working/mymodel')

# Use the model to make predictions
pred = model.predict(img)

# Display the predicted class
predicted_class = tf.keras.applications.resnet50.decode_predictions(pred, top=1)[0][0]
print('Predicted class:', predicted_class[1])


In [ ]:
''''''''''early_stopping_cb = callbacks.EarlyStopping(monitor="val_loss", patience=3)
reduce_lr = callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                              factor=0.2, 
                                              patience=2,
                                              verbose=1, 
                                              min_lr=1e-7)

In [ ]:
'''''''''# Fine-tune the model
# Optionally, you can choose to fine-tune some of the lower layers of the Inception model
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(train_generator,
                    steps_per_epoch=train_generator.n // 40,
                    epochs=10,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.n // 40)